## Mastery grading with core vs. supplemental learning targets

I need this distinction for its own grader since the core vs. supplemental is quite different for its data structure that I don't want to cram it on the calculus grader (which itself depends on 'webassign grade' which is a whole other thing)

Read in the data from the Blackboard exported CSV. 


**Future note** if blackboard columns change, this will be rather broken.

If you're just playing with this, you can read *test.csv* for test data.

In [17]:
import pandas as pd

filename_change_if_youre_testing = "321-lastweek.csv"
midtermOrFinal = "final"

df = pd.read_csv(filename_change_if_youre_testing)

math321_better_cols = ['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability', 
                       'O1', 'O2 Core', 'O3', 'O4 Core', 'O5',
                       'L1 Core', 'L2', 'L3', 'L4', 'L5 Core',
                       'N1 Core', 'N2 Core', 'N3', 'N4', 'N5', 'N6',
                       'R1 Core', 'R2', 'R3', 'R4 Core',
                       'C1 Core', 'C2 Core', 'C3', 'C4']


better_cols = math321_better_cols

df.columns = better_cols

## Make a better index column and dump everything not grade-related
df['Name'] = df['Last Name'] + ', ' + df['First Name']
df.drop(['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability'], axis=1, inplace=True)
df.set_index("Name", inplace=True)

df.replace(r'\s*🎉\s*', '', regex=True, inplace=True)

# Playing with this - it selects only the Core learning targets
core_cols = df.filter(like="Core", axis=1).columns

# this extracts the difference in a roundabout way
# {all columns} \ {regex(core)} = {columns we want}

supplemental_cols = df.filter(set(better_cols).difference(core_cols)).columns


In [18]:
# Aggregate data into P and PM's 
countProficientAndMastered = lambda x: x.value_counts()

coredf = df[core_cols].agg(countProficientAndMastered, axis=1)

supplementaldf = df[supplemental_cols].agg(countProficientAndMastered, axis=1)

coredf.rename(columns = {"P":"Core Proficient", "PM": "Core Mastered"},
             inplace=True)
supplementaldf.rename(columns = {"P":"Supplemental Proficient", "PM": "Supplemental Mastered"},
                     inplace = True) 
coredf.fillna(0, inplace=True)
supplementaldf.fillna(0, inplace=True)

coredf.drop("N", axis=1, inplace=True)
supplementaldf.drop("N", axis=1, inplace=True)


# I truly have no idea why "Name" isn't a common index, but I need to specify it.
df = pd.merge(df, coredf, left_on="Name", right_on="Name", how="inner")
df = pd.merge(df, supplementaldf, left_on="Name", right_on="Name", how="inner")

In [19]:
import collections

DiscreteIndividual = collections.namedtuple('Individual', ['Name', 'CoreMarks', 'SupplementMarks', 'RawData'])
DiscreteMarks = collections.namedtuple('Marks', ['Mastered', 'Proficient'])

## Key      |   Data
## ---------+---------
## Name     |  full name:  last, first
## CoreMarks    |  a namedtuple of type Marks on core target  (Mastered, and Proficient-ed)
## SupplementMarks | namedtuple of marks on supplemental learning targets
## Raw Data |  full pandas Series whose index is the Name above


In [20]:
class_scores = list()
for index, row in df.iterrows():

    coreMarks = DiscreteMarks(row["Core Mastered"],
                              row["Core Proficient"] + row["Core Mastered"])
    supplementMarks = DiscreteMarks(row["Supplemental Mastered"], 
                                    row["Supplemental Proficient"] + row["Supplemental Mastered"])
    
    row = row.drop(["Core Mastered", "Core Proficient", "Supplemental Mastered", "Supplemental Proficient"])
    raw = pd.DataFrame({'Standard':row.index, 'Marks':row.values})
    raw = raw.dropna()
    class_scores.append( DiscreteIndividual(Name=index, 
                                            CoreMarks=coreMarks,
                                            SupplementMarks=supplementMarks,
                                            RawData=raw) )
    


In [21]:

midterm_321 = ( 6, 4, 4, #A 
               4, 2, 2, 6, #B
               3, 3, 2, 4, #C
               3, 4, #D
               3) #F

final_321 = ( 10, 8, 6, #A 
               7, 3, 6, 8, #B
               5, 5, 6, 6, #C
               5, 7, #D
               5) #F


discrete_threshold_table = """<table class='paleGrayRows'> <thead> <tr> 
<th>Letter Grade</th> 
<th>Core Learning Targets</th> 
<th>Supplemental Targets</th> </tr> 
</thead> 
<tbody> 
<tr> <td>A</td> <td>Master %d </td> <td>Master %d, Proficient %d</td> </tr> 
<tr> <td>B</td> <td>Master %d, Proficient %d</td><td>Master %d, Proficient %d</td> </tr> 
<tr> <td>C</td> <td>Master %d, Proficient %d</td> <td>Master %d, Proficient %d</td> </tr> 
<tr> <td>D</td> <td>Proficient %d</td> <td>Proficient %d</td> </tr> 
<tr> <td>F</td> <td>Proficient &lt; %d</td> <td>----</td> </tr> 
</tbody> </table>"""

# I'm an idiot an still have this midterm vs final; oh well
threshold_table = midterm_321 if midtermOrFinal == "midterm" else final_321 





def decide_letter_grade(individual):
    threshold = {"A": {"Core Mastered": threshold_table[0], "Core Proficient": 0,
                     "Supplemental Mastered": threshold_table[1], "Supplemental Proficient": threshold_table[1]+threshold_table[2]-1},
              "B": {"Core Mastered": threshold_table[3], "Core Proficient": threshold_table[3] + threshold_table[4],
                     "Supplemental Mastered": threshold_table[5], "Supplemental Proficient": threshold_table[5]+threshold_table[6]-1},
              "C": {"Core Mastered": threshold_table[7], "Core Proficient": threshold_table[7] + threshold_table[8],
                     "Supplemental Mastered": threshold_table[9], "Supplemental Proficient": threshold_table[9]+threshold_table[10]-1},
              "D": {"Core Mastered": 0, "Core Proficient": threshold_table[11],
                     "Supplemental Mastered": 0, "Supplemental Proficient": threshold_table[12]},
              "F": {"Core Mastered": 0, "Core Proficient": 0,
                     "Supplemental Mastered": 0, "Supplemental Proficient": 0},
          }
    
    
    if individual.CoreMarks.Mastered >= threshold['A']["Core Mastered"] and individual.CoreMarks.Proficient >= threshold["A"]["Core Proficient"] and individual.SupplementMarks.Mastered >= threshold['A']["Supplemental Mastered"] and individual.SupplementMarks.Proficient >= threshold["A"]["Supplemental Proficient"]:
        return "A"
    elif individual.CoreMarks.Mastered >= threshold['B']["Core Mastered"] and individual.CoreMarks.Proficient >= threshold["B"]["Core Proficient"] and individual.SupplementMarks.Mastered >= threshold['B']["Supplemental Mastered"] and individual.SupplementMarks.Proficient >= threshold["B"]["Supplemental Proficient"]:
        return "B"
    elif individual.CoreMarks.Mastered >= threshold['C']["Core Mastered"] and individual.CoreMarks.Proficient >= threshold["C"]["Core Proficient"] and individual.SupplementMarks.Mastered >= threshold['C']["Supplemental Mastered"] and individual.SupplementMarks.Proficient >= threshold["C"]["Supplemental Proficient"]:
        return "C"
    elif individual.CoreMarks.Mastered >= threshold['D']["Core Mastered"] and individual.CoreMarks.Proficient >= threshold["D"]["Core Proficient"] and individual.SupplementMarks.Mastered >= threshold['D']["Supplemental Mastered"] and individual.SupplementMarks.Proficient >= threshold["D"]["Supplemental Proficient"]:
        return "D"
    elif individual.SupplementMarks.Mastered + individual.SupplementMarks.Proficient >= threshold["F"]["Supplemental Proficient"]:
        return "F"
    else:
        return "None of the above?!?"

In [22]:
from jinja2 import Environment, FileSystemLoader
from weasyprint import HTML
from datetime import date

def print_the_report_for(individual):
    today = date.today().strftime("%B %d, %Y")
    env = Environment(loader=FileSystemLoader('.'))
    template = env.get_template("template_core.html")
    template_vars = {
        "title": "Math 321 " + midtermOrFinal + " Grade Report",
        "Name": individual.Name,
        "raw_data": individual.RawData.to_html(index=False, bold_rows=False, classes=["paleGrayRows"]),
        "core_mastered": individual.CoreMarks.Mastered,
        "core_proficient": individual.CoreMarks.Proficient,
        "supplemental_mastered": individual.SupplementMarks.Mastered,
        "supplemental_proficient": individual.SupplementMarks.Proficient,
        "letter_grade": decide_letter_grade(individual),
        "date": today,
        "midterm_or_final": midtermOrFinal,
        "grade_threshold": threshold_html_table,
        "extra_text": extra_text if extra_text else ""
    }
  
    output = template.render(template_vars)
    HTML(string=output).write_pdf("./reports/"+individual.Name+".pdf",stylesheets=["./report.css"])
   

In [23]:
threshold_html_table = discrete_threshold_table % threshold_table
#for individual in class_scores:

# extra_text = "This is a check-in for your progress as of the middle of the semester. For the midterm, I'm reporting only the plain letter grade, no +/-.  Remember this is only to <i>help you</i> see where you stand at this point so you can know where to focus going forward.<br/><br/> Remember that you need to get at all core learning targets (such as L5) correct at least once in order to get a C or better."
extra_text = "<h2>Almost the end of classes</h2><p> <b>Note:</b> This is being prepared for the last week of classes, up to and including checkpoint 14.  Note also the grade thresholds that must be met on the next page.</p> <p> Unhappy with your grade? There's still opportunities on Checkpoint 15 this week and 16 during the finals week. </p> <p>Happy with your grade? High five 🙌. You're done 🎉. Your grade will not decrease. If you do nothing else, this is your final grade.</p>"
#extra_text = "After checkpoint 15"
# extra_text = "This is the final grade report. Letter grade with any plus/minus will be reported to MyWSU. Please don't email me about grades."

#for student in class_scores:
#    print_the_report_for(student)

print(class_scores[5].RawData)

print_the_report_for(class_scores[5])

   Standard Marks
0        O1    PM
1   O2 Core    PM
2        O3    PM
3   O4 Core    PM
4        O5    PM
5   L1 Core    PM
6        L2    PM
7        L3    PM
8        L4    PM
9   L5 Core    PM
10  N1 Core    PM
11  N2 Core    PM
12       N3    PM
13       N4    PM
14       N5    PM
15       N6    PM
16  R1 Core    PM
17       R2    PM
18       R3    PM
19  R4 Core    PM
20  C1 Core    PM
21  C2 Core     P
22       C3    PM
23       C4     P
